In [2]:
import napari
import numpy as np
from skimage.io import imsave
from skimage.util import img_as_uint

In [3]:
# image = np.zeros((50, 50, 50), dtype=float)
# multiscales = nz.to_multiscales(image, scale_factors=[])
# nz.to_ngff_zarr('example.ome.zarr', multiscales)

In [4]:
image_0 = np.zeros((50, 50, 50), dtype=np.float32)
image_0[15:30, 15:35, 15:35] = 0.5
image_0[20:30, 18:28, 18:32] = 0

In [5]:
translate = (10.5, 8.5, 8.5)

image_1 = np.zeros((20, 20, 20), dtype=np.float32)
image_1[5:10, 5:10, 5:12] = 1

In [6]:
imsave("volume_no_transform.tif", img_as_uint(image_0))
imsave("volume_translate_10_5_8_5_8_5_scale_2_2_2.tif", img_as_uint(image_1))

In [7]:
viewer = napari.Viewer()
viewer.add_image(image_0, rendering="iso", colormap="bop blue", blending="translucent")
viewer.add_image(image_1, rendering="iso", colormap="bop orange", blending="translucent")

# translate = (3, 2, 2)
# viewer.add_image(
#     image_1,
#     rendering="iso",
#     colormap="bop orange",
#     blending="translucent",
#     translate=translate,
#     scale=(5, 5, 5),
#     name="image_1 transformed"
# )

<Image layer 'image_1' at 0x32398c2b0>

In [8]:
viewer.add_image(
    image_1,
    rendering="iso",
    colormap="bop orange",
    blending="additive",
    translate=translate,
    scale=(2, 2, 2),
    name="image_1 transformed",
)

<Image layer 'image_1 transformed' at 0x324fedde0>

In [9]:
translation = np.array(
    [
        [0, 0, 0, 10.5],
        [0, 0, 0, 8.5],
        [0, 0, 0, 8.5],
        [0, 0, 0, 0],
    ]
)

scale = np.array(
    [
        [2, 0, 0, 0],
        [0, 2, 0, 0],
        [0, 0, 2, 0],
        [0, 0, 0, 1],
    ]
)

In [10]:
transform = translation + scale
transform

array([[ 2. ,  0. ,  0. , 10.5],
       [ 0. ,  2. ,  0. ,  8.5],
       [ 0. ,  0. ,  2. ,  8.5],
       [ 0. ,  0. ,  0. ,  1. ]])

In [11]:
viewer.add_image(
    image_1, rendering="iso", colormap="bop orange", blending="additive", affine=transform, name="image_1 transformed"
)

<Image layer 'image_1 transformed [1]' at 0x325282890>

In [12]:
from spatialdata.transformations import Scale, Sequence, Translation

/opt/miniconda3/envs/ome/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [14]:
scale = Scale([2, 2, 2], axes=("z", "x", "y"))
scale

Scale (z, x, y)
    [2. 2. 2.]

In [15]:
translation = Translation([10.5, 8.5, 8.5], axes=("z", "y"))
translation

Translation (z, y, x)
    [10.5  8.5  8.5]

In [18]:
sequence = Sequence([scale, translation])
sequence

Sequence 
    Scale (z, x, y)
        [2. 2. 2.]
    Translation (z, y, x)
        [10.5  8.5  8.5]

In [24]:
sequence.to_affine_matrix(input_axes=("z", "y", "x"), output_axes=("z", "y", "x"))

array([[ 2. ,  0. ,  0. , 10.5],
       [ 0. ,  2. ,  0. ,  8.5],
       [ 0. ,  0. ,  2. ,  8.5],
       [ 0. ,  0. ,  0. ,  1. ]])

In [32]:
import json

print(json.dumps(sequence.to_ngff(input_axes=("z", "y", "x"), output_axes=("z", "y", "x")).to_dict(), indent=4))

{
    "type": "sequence",
    "transformations": [
        {
            "type": "scale",
            "scale": [
                2.0,
                2.0,
                2.0
            ],
            "input": {
                "name": "zyx",
                "axes": [
                    {
                        "name": "z",
                        "type": "space",
                        "unit": "unit"
                    },
                    {
                        "name": "y",
                        "type": "space",
                        "unit": "unit"
                    },
                    {
                        "name": "x",
                        "type": "space",
                        "unit": "unit"
                    }
                ]
            },
            "output": {
                "name": "global",
                "axes": [
                    {
                        "name": "z",
                        "type": "space",
                        "uni

In [36]:
from spatialdata.datasets import blobs

sdata = blobs()
sdata

/opt/miniconda3/envs/ome/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/Users/macbook/embl/projects/basel/spatialdata/src/spatialdata/models/models.py:1027: UserWarning: Converting `region_key: region` to categorical dtype.
  return convert_region_column_to_categorical(adata)


SpatialData object
├── Images
│     ├── 'blobs_image': DataArray[cyx] (3, 512, 512)
│     └── 'blobs_multiscale_image': DataTree[cyx] (3, 512, 512), (3, 256, 256), (3, 128, 128)
├── Labels
│     ├── 'blobs_labels': DataArray[yx] (512, 512)
│     └── 'blobs_multiscale_labels': DataTree[yx] (512, 512), (256, 256), (128, 128)
├── Points
│     └── 'blobs_points': DataFrame with shape: (<Delayed>, 4) (2D points)
├── Shapes
│     ├── 'blobs_circles': GeoDataFrame shape: (5, 2) (2D shapes)
│     ├── 'blobs_multipolygons': GeoDataFrame shape: (2, 1) (2D shapes)
│     └── 'blobs_polygons': GeoDataFrame shape: (5, 1) (2D shapes)
└── Tables
      └── 'table': AnnData (26, 3)
with coordinate systems:
    ▸ 'global', with elements:
        blobs_image (Images), blobs_multiscale_image (Images), blobs_labels (Labels), blobs_multiscale_labels (Labels), blobs_points (Points), blobs_circles (Shapes), blobs_multipolygons (Shapes), blobs_polygons (Shapes)

In [47]:
affine = sequence.to_affine(input_axes=("z", "y", "x"), output_axes=("z", "y", "x"))

In [48]:
affine

Affine (z, y, x -> z, y, x)
    [ 2.   0.   0.  10.5]
    [0.  2.  0.  8.5]
    [0.  0.  2.  8.5]
    [0. 0. 0. 1.]

In [49]:
sequence

Sequence 
    Scale (z, x, y)
        [2. 2. 2.]
    Translation (z, y, x)
        [10.5  8.5  8.5]

In [43]:
sdata

SpatialData object
├── Images
│     ├── 'blobs_image': DataArray[cyx] (3, 512, 512)
│     └── 'blobs_multiscale_image': DataTree[cyx] (3, 512, 512), (3, 256, 256), (3, 128, 128)
├── Labels
│     ├── 'blobs_labels': DataArray[yx] (512, 512)
│     └── 'blobs_multiscale_labels': DataTree[yx] (512, 512), (256, 256), (128, 128)
├── Points
│     └── 'blobs_points': DataFrame with shape: (<Delayed>, 4) (2D points)
├── Shapes
│     ├── 'blobs_circles': GeoDataFrame shape: (5, 2) (2D shapes)
│     ├── 'blobs_multipolygons': GeoDataFrame shape: (2, 1) (2D shapes)
│     └── 'blobs_polygons': GeoDataFrame shape: (5, 1) (2D shapes)
└── Tables
      └── 'table': AnnData (26, 3)
with coordinate systems:
    ▸ 'global', with elements:
        blobs_image (Images), blobs_multiscale_image (Images), blobs_labels (Labels), blobs_multiscale_labels (Labels), blobs_points (Points), blobs_circles (Shapes), blobs_multipolygons (Shapes), blobs_polygons (Shapes)

In [44]:
from spatialdata.models import Labels3DModel

In [46]:
x = Labels3DModel.parse(image_0)

INFO     no axes information specified in the object, setting `dims` to: ('z', 'y', 'x')                           


In [51]:
from spatialdata.transformations import set_transformation

set_transformation(x, {"transformation0": affine, "transformation1": sequence}, set_all=True)
x

<xarray.DataArray 'image' (z: 50, y: 50, x: 50)> Size: 500kB
dask.array<array, shape=(50, 50, 50), dtype=float32, chunksize=(50, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * z        (z) float64 400B 0.5 1.5 2.5 3.5 4.5 ... 45.5 46.5 47.5 48.5 49.5
  * y        (y) float64 400B 0.5 1.5 2.5 3.5 4.5 ... 45.5 46.5 47.5 48.5 49.5
  * x        (x) float64 400B 0.5 1.5 2.5 3.5 4.5 ... 45.5 46.5 47.5 48.5 49.5
Attributes:
    transform:  {'transformation0': Affine (z, y, x -> z, y, x)\n    [ 2.   0...

In [53]:
sdata["x"] = x
sdata

/Users/macbook/embl/projects/basel/spatialdata/src/spatialdata/_core/_elements.py:79: UserWarning: Key `x` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


SpatialData object
├── Images
│     ├── 'blobs_image': DataArray[cyx] (3, 512, 512)
│     └── 'blobs_multiscale_image': DataTree[cyx] (3, 512, 512), (3, 256, 256), (3, 128, 128)
├── Labels
│     ├── 'blobs_labels': DataArray[yx] (512, 512)
│     ├── 'blobs_multiscale_labels': DataTree[yx] (512, 512), (256, 256), (128, 128)
│     └── 'x': DataArray[zyx] (50, 50, 50)
├── Points
│     └── 'blobs_points': DataFrame with shape: (<Delayed>, 4) (2D points)
├── Shapes
│     ├── 'blobs_circles': GeoDataFrame shape: (5, 2) (2D shapes)
│     ├── 'blobs_multipolygons': GeoDataFrame shape: (2, 1) (2D shapes)
│     └── 'blobs_polygons': GeoDataFrame shape: (5, 1) (2D shapes)
└── Tables
      └── 'table': AnnData (26, 3)
with coordinate systems:
    ▸ 'global', with elements:
        blobs_image (Images), blobs_multiscale_image (Images), blobs_labels (Labels), blobs_multiscale_labels (Labels), blobs_points (Points), blobs_circles (Shapes), blobs_multipolygons (Shapes), blobs_polygons (Shapes)
    ▸ '

In [54]:
sdata.write("/Users/macbook/Desktop/data.zarr")

INFO     The Zarr backing store has been changed from None the new file path: /Users/macbook/Desktop/data.zarr     
